<a href="https://colab.research.google.com/github/imkunals726/ML/blob/master/MNIST_Using_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
# tf.compat.v1.disable_eager_execution()

In [2]:
tf.__version__

'2.2.0-rc3'

In [0]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(mnist['data'] , mnist['target'])

In [0]:
def input_train_fn(X,y, feature_cols, batch_size=250):
    features ={}
    for idx,col in enumerate(feature_cols):
        features[col] = X[:,idx]
        
    labels =tf.convert_to_tensor(y, dtype=tf.int32)
    
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    return dataset

In [0]:
def train_input_fn():
    return input_train_fn(X_train, y_train, mnist['feature_names'])

In [0]:
feature_columns = [tf.feature_column.numeric_column(col) for col in mnist['feature_names']]

In [12]:
classifier = tf.estimator.DNNClassifier(hidden_units=[300, 100], n_classes=10,
                                             feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpo9wsrsmh', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [13]:

classifier.train(input_fn=train_input_fn, steps=5000)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpo9wsrsmh/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 128.81905, step = 0
INFO

In [0]:
def input_test_fn(X, feature_cols, batch_size=250):
    features ={}
    for idx,col in enumerate(feature_cols):
        features[col] = X[:,idx]
        
    # labels =tf.convert_to_tensor(y, dtype=tf.int32)
    
    dataset = tf.data.Dataset.from_tensor_slices((dict(features)))
    return dataset.batch(batch_size)

In [0]:
preds = classifier.predict(input_fn=lambda : input_test_fn(X_test,feature_columns))   

In [30]:
dnn_preds = []
for prediction in preds:
  dnn_preds.append(prediction['class_ids'][0])

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpo9wsrsmh/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [0]:
from sklearn.metrics import accuracy_score
import numpy as np

In [37]:
accuracy_score( y_test.astype(int),np.array(dnn_preds) )

0.9236571428571428

In [0]:
def evaluate_input_fn():
    return input_train_fn(X_test, y_test, mnist['feature_names'])

In [0]:
classifier.evaluate(input_fn=evaluate_input_fn)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-04-26T09:37:12Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpo9wsrsmh/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
